In [97]:
import pandas as pd
import numpy as np

In [98]:
def get_daily_cases(data, col_name='series_1'):
    series = data.copy()
    series = series.groupby(by='Date_statistics').count()
    series = series.rename(columns={'Date_file': col_name})
    return series[col_name]


def make_measures_0_1(column):
    column=np.where(column==daily_data[column.name].max(),column, 0)
    column=np.where(column==0,column, 1)
    return column

In [99]:
#Load Data
measures = pd.read_excel('measures_strictness.xlsx', engine='openpyxl').rename(columns={'Unnamed: 0':'Date'})
cases_national= pd.read_csv('Data/COVID-19_casus_landelijk.csv', sep=';')
behaviour_data= pd.read_csv('Data/COVID-19_gedrag.csv', sep=';')

In [100]:
# Create datasets

# Put date in the right format, merge with measures data
cases_national['Date_statistics'] = pd.to_datetime(cases_national['Date_statistics'],errors='ignore', dayfirst=True)
daily_cases = get_daily_cases(cases_national, 'Number of new cases').to_frame().reset_index()
daily_cases['Date_statistics'] = pd.to_datetime(daily_cases['Date_statistics'])
daily_data = daily_cases.merge(measures, left_on='Date_statistics', right_on='Date', how='left').drop(['Date', 'stringency_index', 'government_response_index', 'containment_health_index', 'economic_support_index'], axis=1)

# Put date in the right format, filter to simplify the dataset, drop useless columns
behaviour_data['Date_of_measurement'] = pd.to_datetime(behaviour_data['Date_of_measurement'],errors='ignore')
behaviour_data = behaviour_data.loc[lambda d: (d.Region_name=='Nederland') & (d.Subgroup == 'Totaal') & (d.Sample_size != 0) & ((d.Indicator_category.str.contains('Naleving')) | (d.Indicator_category.str.contains('Zorgen')))]
behaviour_data= behaviour_data.drop(['Date_of_report', 'Wave','Region_code', 'Subgroup', 'Figure_type', 'Region_name', 'Subgroup_category', 'Lower_limit', 'Upper_limit', 'Change_wrt_previous_measurement'], axis=1)
behaviour_data['Normalised_Value'] = behaviour_data['Value'] / behaviour_data['Sample_size']

In [101]:
new=daily_data.drop(['Date_statistics','Number of new cases'],axis=1).apply(make_measures_0_1,axis=0,result_type='expand')
daily_data=pd.concat([daily_data['Date_statistics'],daily_data['Number of new cases'],new],axis=1)
daily_data.to_csv('daily_data.csv',index=False)
behaviour_data.to_csv('behaviour_data.csv',index=False)